# Workshop 12

# Plan for today:
- A model project example
- Work on your model project

# From Malthusian stagnation to sustained economic growth

The Malthus hypothesis states that in pre-industrial times, increases in technological sophistication lead to an increase in population and population density in the short- and long-run, but do not permanently raise income per capita levels. In this assignment, we rely on a model by [Ashraf and Galor (2011)](https://www.aeaweb.org/articles?id=10.1257/aer.101.5.2003) and show that when utility depends only on consumption and the number of children, this hypothesis indeed holds true. Even when we allow technology to be endogenous to the population, this continues to be the case. However, when we extent the model such that utility also depends on the education level of the offspring, sustained economic growth can be achieved. This is because households face a quantity-quality tradeoff, where quantity refers to the number of children and quality refers to the education level of children. Increases in the rate of technological prgoress induce a fertility decline as households optimize their utility by reducing the number of offspring, but increasing their level of education.

**Imports and set magics:**

In [1]:
import numpy as np
import scipy as sp
from scipy import optimize
import sympy as sm
import math
from math import log

import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual
# autoreload modules when code is run
%load_ext autoreload
%autoreload 2

# local modules
from modelproject import *
import matplotlib.pyplot as plt
plt.style.use('seaborn-whitegrid')


# The Malthusian Model: Model Description

We consider the following production function:


$$Y_t = (AX)^\alpha L_t^{1-\alpha}\label{eq1}\tag{1}$$

where

* $Y_t$ is output in period t
* $L_t$ is population in period t (growing at rate n)
* $A$ is technology (fixed)
* $X$ is land (fixed)

Hence, output per capita is defined as:

$$Y_t/L_t = y_t = (AX/L_t)^\alpha\label{eq2}\tag{2}$$

implying decreasing returns to land, i.e., $\frac{\partial y}{\partial L}<0$


We assume that household live for two periods. In period 2, they work and receive income y which they can spend on either conumption (c) or having children (n). Household preferences are as follows:

$$U(c_t, n_t)=(1-\gamma) ln(c_t) + \gamma ln(n_t)\label{eq3}\tag{3}$$

The budget constraint is given by:

$$y_t = c_t + \rho n_t\label{eq4}\tag{4}$$

where $\rho$ is the price of having children relative to consumption

## Solving the Model

### We start by optimizing the household's utility subject to the budget constraint
We start out by defining the model parameters and variables using `sympy`. Furthermore, we write up the utility function, $U(c_t,n_t)$, and define the steady states.

In [2]:
# a. variables
Y = sm.symbols('Y')
A = sm.symbols('A')
L = sm.symbols('L')
y = sm.symbols('y')
X = sm.symbols('X')
U = sm.symbols('U')
c = sm.symbols('c')
n = sm.symbols('n')
lt_1= sm.symbols('L_{t+1}')
lt= sm.symbols('L_t')

# b. parameters
alpha = sm.symbols('alpha')
gamma = sm.symbols('gamma')
rho = sm.symbols('rho')

# c. steady states
pstar = sm.symbols('P^\star')
lstar = sm.symbols('L^\star')
ystar = sm.symbols('y^\star')
ustar = sm.symbols('U^\star')

# d. utility function
U = (1-gamma)*sm.ln(c)+gamma*sm.ln(n)
sm.Eq(ustar,U)

Eq(U^\star, gamma*log(n) + (1 - gamma)*log(c))

***
**Next, we determine how households optimally divide their income between consumption and having offspring**
*** 
    
After doing the preliminary work with defining the variables and parameters as well as the utility function we create an object, $t1$, from the class 'Modelproject' and the household problem using the parameters stated in the class:

In [3]:
t1 = Modelproject() # calls the class
t1.solve() # solves the model
print(f' (1-gamma) %: {round(t1.c,3)}') # displays optimal consumption
print(f' gamma/rho %: {round(t1.n,3)}') # displays the optimal number of children

 (1-gamma) %: 30.001
 gamma/rho %: 63.636


We find that housholds optimally spend fraction $(1-\gamma) = 30.00$ of their income on consumption, and fraction $\gamma/\rho = 63.636$ on offspring. 

### In the next step, we are determining the population and population density in steady state

The population evolves as follows:

$$L_{t+1} = n_t L_t\label{eq5}\tag{5}$$

Inserting the optimality condition for population growth, $n_t$, into the equation yields:

$$L_{t+1} = \frac{\gamma}{\rho}(AX)^\alpha L_t^{1-\alpha}\label{eq6}\tag{6}$$

Using (\ref{eq6}) we solve for the population in steady state by setting $L_{t+1}=L_t=L$ and isolate $L$:

***
**Population and population density in steady state (analytical solution)**
***

In [4]:
# a. define equation (6) in steady state, i.e., L_t+1 = L_t = L
ss = sm.Eq(L, (((gamma/rho)*(A*X)**alpha)*(L**(1-alpha))))

# b. isolate L and print population in steady state
pss = sm.solve(ss, L)[0]
sm.Eq(lstar,pss)

Eq(L^\star, (gamma*(A*X)**alpha/rho)**(1/alpha))

Population density is defined as $P_{t+1}=\frac{L_{t+1}}{X} = \frac{\gamma}{\rho}(A)^\alpha \cdot (XL_t)^{1-\alpha} \Rightarrow P^*=\frac{L^*}{X}$. 

We therefore divide the steady state population with $X$ and write up the population density in steady state, $P^*$:

In [5]:
# a. define population density in steady state
P = pss/X

# b. determine the steady state level of population density
sm.Eq(pstar,P)

Eq(P^\star, (gamma*(A*X)**alpha/rho)**(1/alpha)/X)

***
**Population and population density in steady state (numerical solution)** 
*** 
We now want to find the population number in steady state. We use the bisection algorithm which is a derivative free method of finding the roots of a function, i.e. $x_0\in[a,b]$ such that $f(x_0)=0$. The logic behind the algorithm is such that we start with a big interval that we know contains the desired value, $x_0$. We then reduce this interval successively. If the function, $f$ is continuous and takes on values between $f(a)$ and $f(b)$ at the lower and upper bound respectively, then $f$ will take on values between $f(a)$ and $f(b)$ in the interval $[a,b]$. Hence, if $f(a)<\gamma<f(b)$ then it must be the case that there exists some $c\in[a,b]$ such that $f(c)=\gamma$.

We start off by ensuring that the function value evaluated at each of the two bounds are not the same. If this were not the case then the interval would not contain a root.

The algorithm goes as follows:
1. Set $a_0=a$ and $b_0=b$ and make sure that $f(a_0)$ and $f(b_0)$ have opposite signs. 
2. Compute the function value at the midpoint: $f(m_0)$ where $m_0=\frac{a_0+b_0}{2}$ is the midpoint
3. Determine the sub-intervals:
* If $\text{sign}(f(a_0)) ≠ \text{sign}f(m_0)$ then $a_1=a_0$ and $b_1=m_0$.
* If $\text{sign}(f(m_0)) ≠ \text{sign}f(b_0)$ then $a_1=m_0$ and $b_1=b_0$.
4. Repeat step 2 and 3 until $f(m_n)<\varepsilon$.

In [11]:
# a. upper and lower limit have same sign
def samesign(a, b):
        return a * b > 0


# b. create the bisection algorithm
def bisect(func, low, high):
    # set tolerance level
    tol = 1e-6
    
    # step 1: Assertain that the function evaluated at the lower and upper bound respectively have different signs
    assert not samesign(func(low), func(high))
    
    # step 2: Compute the mid-point 
    for i in range(100):
        midpoint = (low + high) / 2.0
        fm=func(midpoint)
        # step 3: Determine sub-intervals
        if abs(fm) < tol:
            break
        elif samesign(func(low), func(midpoint)):
            low = midpoint
        else:
            high = midpoint
    return midpoint

# d. define function and the parameter values
def f(L, gamma=0.7, rho=1.1, A=25, X=10, alpha=1/3):
        return L-((gamma/rho)*(A*X)**alpha)*(L**(1-alpha))

# e. run the algorithm with f being the function, the lower bound being 0.1 and the upper bound being 1000
L = bisect(f, 0.1, 1000)
print(f'Steady state level of population for given parameter values is: L*=', round(L,3))

# f. find the population density in steady state X= 1
print(f'The population density in steady state for given parameter values is {round(L/10,3)}')

Steady state level of population for given parameter values is: L*= 64.425
The population density in steady state for given parameter values is 6.443


Hence, we find that the steady state level of labor is 64.425 and a population density of 6.443.

We can do the same as above using `scipy` and arrive at the same result:

In [7]:
# a. call bisect optimzer from .py file and print population steady state
print(f'Steady state population for given parameter values is: {bisect_ss_l(0.1,500):.3f}')

# b. find the population density in steady state
print(f'The population density in steady state for given parameter values is {bisect_ss_l(0.1,500)/10:.3f}')

Steady state population for given parameter values is: 64.425
The population density in steady state for given parameter values is 6.443


Hence, we find, again that the steady state level of population for given the parameter values is 64.425 and that the population density is 6.443.

## Next, we are going to analyze the impact of an exogenous increase in the technology level $A$ on population and population density in steady state

In [8]:
# a. differentiate population in steady state with respect to A
pss_diff = sm.diff(pss, A)

# b. display differentiated population
pss_diff

(gamma*(A*X)**alpha/rho)**(1/alpha)/A

We can see that an increase in A increases the steady state population.

In [9]:
# a. differentiate population density in steady state with respect to A
P_diff = sm.diff(P,A)

# b. display differentiated population density
P_diff

(gamma*(A*X)**alpha/rho)**(1/alpha)/(A*X)

We can see that an increase in A also increases steady state population density.

## In the following, we create a plot of the Malthus model

In [10]:
simulate_malthus("pop")

interactive(children=(FloatSlider(value=0.7, description='$\\gamma$', max=0.99, step=0.05), FloatSlider(value=…